In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True, connected=True)

import json
from tqdm.notebook import tqdm; tqdm.pandas();

import duckdb

import sys
import os
from glob import glob
import gc
sys.path.append('..')
from config import CFG
CFG = CFG()

from src.utils import *
from src.data_processing.hands import *

In [6]:
%load_ext sql
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [8]:
%sql duckdb:///../data/train.db

In [9]:
s2p_map = {k.lower():v for k, v in read_json_file(CFG.PREDICTION_IDX).items()}
p2s_map = {v:k for k, v in read_json_file(CFG.PREDICTION_IDX).items()}

encoder = lambda x: s2p_map.get(x.lower())
decoder = lambda x: p2s_map.get(x)

train = pd.read_csv(f'{CFG.RAW_DATA_PATH}/extended_train.csv')

DEMO_ROW = 283
demo_sign_df = get_sign_df(train.iloc[DEMO_ROW]['path'])
display(demo_sign_df)

frame            row_id        type  landmark_index         x         y  \
0        23         23-face-0        face               0  0.381393 -0.377334   
1        23         23-face-1        face               1  0.387510 -0.333088   
2        23         23-face-2        face               2  0.384334 -0.349668   
3        23         23-face-3        face               3  0.377555 -0.302792   
4        23         23-face-4        face               4  0.388338 -0.322209   
...     ...               ...         ...             ...       ...       ...   
9226     39  39-right_hand-16  right_hand              16       NaN       NaN   
9227     39  39-right_hand-17  right_hand              17       NaN       NaN   
9228     39  39-right_hand-18  right_hand              18       NaN       NaN   
9229     39  39-right_hand-19  right_hand              19       NaN       NaN   
9230     39  39-right_hand-20  right_hand              20       NaN       NaN   

             z  
0    -0.045009  
1    -0.060799  
2    -0.037500  
3    -0.038101  
4    -0.062246  
...        ...  
9226       NaN  
9227       NaN  
9228       NaN  
9229       NaN  
9230       NaN  

[9231 rows x 7 columns]

In [10]:
train.head(3)

path  participant_id  \
0  /home/broug/Desktop/sign-language/data/raw/tra...           26734   
1  /home/broug/Desktop/sign-language/data/raw/tra...           28656   
2  /home/broug/Desktop/sign-language/data/raw/tra...           16069   

   sequence_id   sign  start_frame  end_frame  total_frames  face_count  \
0   1000035562   blow           20         42            23       10764   
1   1000106739   wait           29         39            11        5148   
2    100015657  cloud          103        207           105       49140   

   face_nan_count  pose_count  ...     z_min     z_max  face_appears_pct  \
0               0         759  ... -2.782624  1.910815               1.0   
1               0         363  ... -2.543970  1.627621               1.0   
2               0        3465  ... -2.838325  1.587503               1.0   

   face_nan_pct  left_hand_appears_pct  left_hand_nan_pct  pose_appears_pct  \
0           0.0                    1.0           1.000000               1.0   
1           0.0                    1.0           1.000000               1.0   
2           0.0                    1.0           0.733333               1.0   

   pose_nan_pct  right_hand_appears_pct  right_hand_nan_pct  
0           0.0                     1.0            0.521739  
1           0.0                     1.0            0.818182  
2           0.0                     1.0            1.000000  

[3 rows x 29 columns]

In [12]:
%%sql 
CREATE TABLE 
    features (
        participant_id int,
        sequence_id int,
        frame int,
        type varchar(255),
        index varchar(255),
        x float(10),
        y float(10),
        z float(10)
    )

""
Loading... (need help?)


In [24]:
def get_seq_data(row, invert_y=True):
    """Calculates and adds metadata to the given row of sign language event data.
    
    Args:
        row (pandas.core.series.Series): A row of sign language event data containing columns:
            path: The file path to the Parquet file containing the landmark data for the event.
        invert_y (bool, optional): Whether to invert the y-coordinate of each landmark. Defaults to True.
    
    Returns:
        pandas.core.series.Series: The input row with added metadata columns:
            start_frame: The frame number of the first frame in the event.
            end_frame: The frame number of the last frame in the event.
            total_frames: The number of frames in the event.
            face_count: The number of landmarks in the 'face' type. [optional]
            pose_count: The number of landmarks in the 'pose' type. [optional]
            left_hand_count: The number of landmarks in the 'left_hand' type. [optional]
            right_hand_count: The number of landmarks in the 'right_hand' type. [optional]
            x_min: The minimum x-coordinate value of any landmark in the event.
            x_max: The maximum x-coordinate value of any landmark in the event.
            y_min: The minimum y-coordinate value of any landmark in the event.
            y_max: The maximum y-coordinate value of any landmark in the event.
            z_min: The minimum z-coordinate value of any landmark in the event.
            z_max: The maximum z-coordinate value of any landmark in the event.
    """
    df = get_sign_df(row[1], invert_y=invert_y)

    df['participant_id'] = row[2]
    df['sequence_id'] = row[3]

    return df

type_kp_map = dict(face=468, left_hand=21, pose=33, right_hand=21)
col_order = [
    'participant_id', 'sequence_id', 'frame', 'type', 'landmark_index', 'x', 'y', 'z'
]
from tqdm import tqdm
for rows in tqdm(train.itertuples()):
    df = get_seq_data(rows)
    df = df[col_order]
    %sql INSERT INTO features SELECT * FROM df
# df = train.iloc[200].progress_apply(lambda x: get_seq_data(x))
# df = df[col_order]
# df
# %sql INSERT INTO features SELECT * FROM df

14353it [1:22:16,  3.15it/s]

In [ ]:
df = train.copy()
df['label'] = df['sign'].apply(encoder)
df = df[['participant_id', 'sequence_id', 'sign', 'label']]

%sql CREATE TABLE labels AS SELECT * FROM df